In [1]:
import pandas as pd
import numpy as np
import dash
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import dash_bootstrap_components as dbc

In [2]:
external_stylesheets = [dbc.themes.JOURNAL]

## Initialize App

In [3]:
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
#uses jupyter version of dash, __name__ is a system variable in python that indicates whether the script is being run directly or via a file 
#if run directly it says main, if file it will be the name of a file. Just tells you where the code is coming from

## Specify Contents of the Dashboard

In [4]:
#replace layout attribute with dashboard items
#two types of webpages - static (doesn't change) and interactive 
app.layout = html.Div(
[
    html.H1("Bill and Ted's Excellent Dashboard"), 
    
    dcc.Input(id = 'username', type = 'text', placeholder = 'Please write your name here'),
    #asking the user to input something - a textbox the user can enter their name into 
    
    dcc.Markdown(id='countletters')
]

)

## Callbacks

In [5]:
#uses a decorator (@) - that has a function a few spaces down - applies it to the very next function/thing that comes up in script
#every callback has a list of outputs that go back to the dashboard first, then list of inputs from the dashboard to the next function
@app.callback(Output(component_id = 'countletters', component_property = 'children'), 
              [Input(component_id = 'username', component_property = 'value')])

#function callback refers to 
def countletters(name):
    n = str(name).replace(' ', '')
    return f"you have {len(n)} letters in your name dawg!"

## Run the App

In [6]:
if __name__== "__main__":
    app.run_server(mode= 'inline', host = "0.0.0.0", debug=True)